In [ ]:
import json

In [ ]:
# データを取得したい領域の座標情報を取得
from IPython.display import HTML
HTML(r'<iframe src="https://www.keene.edu/campus/maps/tool/" width="960" height="480" frameborder="0"></iframe>')

In [ ]:
# サンプルGeoJSONファイルを読み込む
with open("./geojson/sample-aoi.geojson","r") as f:
    dic = json.load(f)
dic["features"][0]["geometry"]["coordinates"] = AREA

# GeoJSONファイルの書き出し
with open("./geojson/result-aoi.geojson", "w") as f:
    json.dump(dic, f)

In [ ]:
# 座標情報を以下に貼り付ける
AREA = [
    [
      [
        139.7694397,
        35.761279
      ],
      [
        139.7642899,
        35.6846293
      ],
      [
        139.8762131,
        35.6751474
      ],
      [
        139.878273,
        35.7568214
      ],
      [
        139.7684097,
        35.762672
      ]
    ]
  ]

In [1]:
import os
from glob import glob
import sys
import tempfile
import imageio
import json
import geojson
import math
import subprocess
from tqdm import tqdm
import numpy as np
from osgeo import gdal, gdal_array

In [ ]:
def get_bounding_box(geometry):
    """
    AOIから対象範囲の緯度経度情報を取得
    
    Parameters
    -----------
    geometry: AOI情報(dict)
    
    Returns
    -----------
    left:   最低経度情報(float)
    right:  最高経度情報(float)
    bottom: 最低緯度情報(float)
    top:    最高緯度情報(float)
    """
    coords = np.array(list(geojson.utils.coords(geometry)))
    
    ## 経度情報 ##
    left   = coords[:, 0].min()
    right  = coords[:, 0].max()
    ## 緯度情報 ##
    bottom = coords[:, 1].min()
    top    = coords[:, 1].max()
    
    return left, right, bottom, top

In [ ]:
def get_tile_coords(longitude, latitude, zoom):
    """
    経度、緯度情報からタイル番号を取得する
    
    Parameters
    -----------
    longitude: 経度(float)
    latitude : 緯度(float)
    zoom     : ズームレベル(int)
    
    Returns
    -----------
    xTile: X軸のタイル番号
    yTile: Y軸のタイル番号
    """
    
    xTile = math.floor((longitude + 180) / 360 * (1 << zoom))
    yTile = math.floor((1 - math.log(math.tan(latitude * math.pi / 180) + \
                        1 / math.cos(latitude * math.pi / 180)) / math.pi) / 2 * (1 << zoom))

    return xTile, yTile

In [ ]:
def fetch_tile(zoom_level, x, y, tmp_dir):
    """
    タイル情報の取得

    Parameters
    -----------
    zoom_level : ズームレベル(int)
    x          : X軸のタイル番号(int)
    y          : Y軸のタイル番号(int)
    tmp_dir    : 各タイルの一時ディレクトリ(string)

    Returns
    --------
    None

    """
    # 標準地図
    url_lists = [f"https://cyberjapandata.gsi.go.jp/xyz/std/{zoom_level}/{x}/{y}.png",f"https://cyberjapandata.gsi.go.jp/xyz/seamlessphoto/{zoom_level}/{x}/{y}.jpg"
                 ,f"https://cyberjapandata.gsi.go.jp/xyz/relief/{zoom_level}/{x}/{y}.png"]
    
    for url in url_lists:
        base_name = url.split('/')[4]
        output_dir = tmp_dir + base_name
        if not(os.path.exists(output_dir)):
            os.makedirs(output_dir)
            
        raster = imageio.imread(url)
        raster = raster.transpose(2, 0, 1)
        nodata = -9999
        
        ### タイル境界線の緯度経度情報を取得
        left, top = get_boundary_dgree(zoom_level, x, y)
        right, bottom = get_boundary_dgree(zoom_level, x+1, y+1)

        ### Geotiffファイルを一時ディレクトリに保存する
        dem_tmp_path = os.path.join(output_dir, f"{base_name}_tile_{left}_{top}_{right}_{bottom}.tif")
        ds = gdal_array.OpenArray(raster)
        options = gdal.TranslateOptions(format="GTiff", outputSRS="EPSG:4326" 
                                        ,outputBounds=[left, top, right, bottom])
        gdal.Translate(dem_tmp_path, ds, options=options)
        
        ds = None
        

In [ ]:
def get_boundary_dgree(zoom_level, xtile, ytile):
    """
    タイル境界線の緯度経度情報を取得

    Parameters
    -----------
    zoom_level: ズームレベル(int)
    xtile     : X軸のタイル番号(int)
    ytile     : Y軸のタイル番号(int)

    Returns
    --------
    longitude: 経度(float)
    latitude : 緯度(float)
    """
    n = 1 << zoom_level
    longitude = xtile / n * 360 - 180
    lat_rad = math.atan(math.sinh(math.pi*(1-2*ytile/n)))
    latitude = math.degrees(lat_rad)
    
    return longitude, latitude

In [ ]:
def fetch_all_tiles(zoom_level, top_tile, left_tile, bottom_tile, right_tile, tmp_dir):
    """
    タイル情報の取得

    Parameters
    -----------
    zoom_level:  ズームレベル(int)
    top_tile:    Y軸で最頂点のタイル番号(int)
    left_tile:   X軸で最も左のタイル番号(int)
    bottom_tile: Y軸で最も底のタイル番号(int)
    right_tile:  X軸で最も右のタイル番号(int)
    tmp_dir:     各タイルの一時ディレクトリ(string)
 
    Returns
    --------
    None

    """
    
    x_range = range(left_tile, right_tile+1)
    y_range = range(top_tile, bottom_tile+1)
    
    for x in tqdm(x_range):
        for y in y_range:
            try:
                fetch_tile(zoom_level, x, y, tmp_dir)
            except:
                continue

In [ ]:
def check_3band(target_dir):
    """
    タイルのバンド数チェック
    バンド数が３より多い場合、バンド数を３つに調整する

    Parameters
    -----------
    target_dir: TIFファイルが格納されているディレクトリパス(string)
 
    Returns
    --------
    None

    """
    
    checkDir_list = glob(target_dir + "*/")
    
    for checkDir in checkDir_list:
        checkFile_list = glob(checkDir + "*.tif")
        
        for checkFile in checkFile_list:
            gdalImage = gdal.Open(checkFile)
            gdalImage_array = gdalImage.ReadAsArray()
            
            if len(gdalImage_array) > 3:
                Xsize, Ysize = gdalImage.RasterXSize, gdalImage.RasterYSize # X, Y方向のピクセル数
                dtype = gdal.GDT_Byte
                band = 3
    
                # 空の出力ファイルを作成
                out = gdal.GetDriverByName('GTiff').Create(checkFile, Xsize, Ysize, band, dtype)
    
                # 出力ファイルの座標系を設定
                out.SetProjection(gdalImage.GetProjection())
                out.SetGeoTransform(gdalImage.GetGeoTransform())

                # Red, Green, Blueバンドの配列を書き込む
                out.GetRasterBand(1).WriteArray(gdalImage_array[0]) # Redバンド
                out.GetRasterBand(2).WriteArray(gdalImage_array[1]) # Greenバンド
                out.GetRasterBand(3).WriteArray(gdalImage_array[2]) # Blueバンド
                out.FlushCache()
            elif len(gdalImage_array) == 3:
                continue
    
            else:
                print("error")

In [ ]:
json_name = "./geojson/sample-aoi.geojson" # AOIのgeojsonファイルパス
output_dir = "./output/" # 出力ディレクトリ
zoom_level = 15 # zoom_level(15固定)

dir_lists = ["std", "seamlessphoto", "relief"]
output_lists = ["03_standard_map", "04_orho", "06_elevation"]

# 入力geojsonの読み込み
with open(json_name, "r") as f:
    dic = json.load(f)
    
# 取得タイルの範囲を取得
geometry = dic["features"][0]["geometry"]
left,right,bottom,top = get_bounding_box(geometry)

# 入力geojsonカバーするタイル番号の取得
left_tile, top_tile = get_tile_coords(left, top, zoom_level)
right_tile, bottom_tile = get_tile_coords(right, bottom, zoom_level)

# タイルの取得
fetch_all_tiles(zoom_level, top_tile, left_tile, bottom_tile, right_tile, output_dir)

# タイルのバンド数を確認
check_3band(output_dir)


for idx, base_dir in tqdm(enumerate(dir_lists)):
    base_name = output_dir.split("/")[-2]
    target_dir = output_dir + base_dir + "/"
    result_dir = "./merge_data/{directory}/".format(directory=output_lists[idx])
    output_name = result_dir + "{filename}.tif".format(filename=base_name)
    filenames = os.listdir(target_dir)
    
    with open("./list.txt", "w", encoding="utf-8") as f:
        for name in filenames:
            root, ext = os.path.splitext(name)
            f.writelines("\"{directory}{filename}\"\n".format(directory=target_dir,
                                                              filename=name))
    if not(os.path.exists(output_dir)):
        os.makedirs(output_dir)
        
    cmd = f"python /opt/anaconda3/bin/gdal_merge.py -o {output_name} --optfile list.txt"
    result = subprocess.call(cmd, shell=True)
    os.remove("list.txt")